In [28]:
#MAKE Simulated sequence networks for evolution experiments to display sequence variation in cytoscape
import pandas as pd
import CommonMotifProb

In [29]:
NTevolveprobDICT1={'A':[1-(1.8*10**-5+1.5*10**-5+2.0*10**-4), 1.8*10**-5, 1.5*10**-5, 2.0*10**-4],
                  'T':[1.4*10**-5, 1-(2.3*10**-4+3.5*10**-5+1.4*10**-5), 2.3*10**-4, 3.5*10**-5],
                  'C':[7.7*10**-6, 2.7*10**-5, 1-(7.7*10**-6+2.7*10**-5+5.1*10**-6), 5.1*10**-6],
                  'G':[3.1*10**-5, 3.5*10**-5, 5.4*10**-5, 1-(3.1*10**-5+3.5*10**-5+5.4*10**-5)]}
NTevolveprobDICT=NTevolveprobDICT1
PDICT={}
nts=['A','T','C','G']
for i in nts:
    for j in range(len(nts)):
        PDICT[i+nts[j]]=NTevolveprobDICT[i][j]
probDICT={}
for i in CommonMotifProb.DNACodontoAADICT:
    for j in CommonMotifProb.DNACodontoAADICT:
        if '-' not in i and '-' not in j:

            probDICT[i+j]=PDICT[i[0]+j[0]]*PDICT[i[1]+j[1]]*PDICT[i[2]+j[2]]

In [30]:
#for each codon at a given motif position eg S/T: compare it with all other codons: If the difference is one, calculate
#the probability either given the normal metric or the experimental mutation rate and save codon1 -> interaction -> codon2

def makeCodonnetwork(codonlist):
    interactList=[['Codon 1','Interaction', 'Codon2']]
    for codon1 in codonlist:
        for codon2 in codonlist:
            if CommonMotifProb.SeqMatch(codon1,codon2)==1:
                interactWeight=probDICT[codon1+codon2]
                interactList.append([codon1,interactWeight,codon2])
    return interactList

In [31]:
#for each codon at a given motif position eg S/T: compare it with all other codons: If the difference is one, calculate
#the probability either given the normal metric or the experimental mutation rate and save codon1 -> interaction -> codon2

def makeSeqnetwork(seqlist):
    interactList=[]
    uniquesource=[]
    for seq1 in seqlist:
        
        uniquesource.append(seq1)
        for seq2 in seqlist:
            if CommonMotifProb.SeqMatch(seq1,seq2)==1:
                interactWeight=1
                for l in range(len(seq1)):
                    interactWeight*=PDICT[seq1[l]+seq2[l]]
                interactList.append([seq1,interactWeight,seq2])
    return interactList

In [32]:
def ismotifmatch(seq,motif):
    for i in range(len(seq)):
        if seq[i] not in motif[i]:
            return False
    return True

In [33]:
def createmotif(motif):
    finalmotif=[]
    for i in motif:
        if i[0]=='.':
            finalmotif.append(list(CommonMotifProb.AminoAcid1LTR))
        elif i[0]=='^':
            print (i)
            aas=i[1:]
            finalmotif.append(list(CommonMotifProb.AminoAcid1LTR-set(aas)))
        else:
            finalmotif.append(i)
    return finalmotif

In [34]:
import pandas as pd
#NeutralSamplingDiffRatesNoBias_forMS1-0.csv
DFlist=[pd.read_csv('Data/NeutralSamplingDiffGenotypesNew_forMS1-'+str(i)+'.csv',index_col=0) for i in range(10)]#,pd.read_csv('PKAmotifsimAltRate500reps_2.csv',index_col=0)]
#DFlist2=[pd.read_csv('NeutralSamplingDiff2AAs_forMS1-'+str(i)+'.csv',index_col=0) for i in range(10)]
df=pd.concat(DFlist)#+DFlist2)


In [35]:

df2=df[(df['StartCodon']=='AGGAGGATATTGAGG')]#|(df['StartCodon']=='AGGAGGATATTGAGG')]
a=df2['TargetCodon'].value_counts()
print( a)

TargetCodon
AGGAGGATATTGAGG    9960
AGGAGGATACTGAGG    9238
AGGAGGACATTGAGG    9212
AGGAGGATATCGAGG    9182
AGGAGGATATTGGGG    8923
                   ... 
AGGAGGAGAGCGAGG       1
AGGAGGATACTGGGC       1
GGGAGGATACTGAGC       1
AGGAGGACAGCGAGG       1
AGGAGGATATGGTGG       1
Name: count, Length: 870, dtype: int64


In [36]:
df

,StartCodon,PopReplicate,TargetCodon,Count
0,CGACGAATCCTCCGA,1,CGACGAATCCTCCGA,11027
1,CGACGAATCCTCCGA,1,CGTCGAATCCTCCGA,30
2,CGACGAATCCTCCGA,1,CGACGAGTCCTCCGA,2
3,CGACGAATCCTCCGA,1,CGACGAATCCGCCGA,1
4,CGACGAATCCTCCGA,1,CGACGAATCCTCCGG,1
...,...,...,...,...
67349,AGGAGGAGTTTGAGG,1000,AGGAGGAGTTTGGGG,2
67350,AGGAGGAGTTTGAGG,1000,AGGAGGAGTTTGAGG,11110
67351,AGGAGGAGTTTGAGG,1000,AGGAGGAGTTTGAGA,4
67352,AGGAGGAGTTTGAGG,1000,AGGAGGAGTTTTAGG,1


In [37]:
from CommonMotifProb import SeqMatch
seqnetworklist=[]
uniquesource=[]
for i in a.keys():
    uniquesource.append(i)
    for j in a.keys():
        if SeqMatch(i,j)==1 and j not in uniquesource:
            seqnetworklist.append([i,j])

In [38]:
seqnetworklist=makeSeqnetwork(a.keys())
columns=['SourceSeq','ProbWeight', 'TargetSeq']
#seqnetworklist

df2=pd.DataFrame(seqnetworklist,columns=columns)

In [39]:
aaseqs=[]
for i,row in df2.iterrows():
    seq1=CommonMotifProb.translateRNA(row.SourceSeq)
    seq2=CommonMotifProb.translateRNA(row.TargetSeq)
    aaseqs.append([seq1,row.ProbWeight,seq2])
    
dfaa=pd.DataFrame(aaseqs,columns=columns)

In [40]:
newaadictdata=[]
alreadychecked=[]
for i,row in dfaa.iterrows():
    if row.SourceSeq+row.TargetSeq not in alreadychecked and row.SourceSeq!=row.TargetSeq and '*' not in row.SourceSeq :

        dftemp=dfaa[(dfaa['SourceSeq']==row.SourceSeq)&(dfaa['TargetSeq']==row.TargetSeq)]
        probs=dftemp['ProbWeight']
        probssum=probs.sum()
        newaadictdata.append([row.SourceSeq, probssum, row.TargetSeq])
        alreadychecked.append(row.SourceSeq+row.TargetSeq)

In [41]:
dfnewaa=pd.DataFrame(newaadictdata,columns=columns)

In [42]:
dfnewaa.to_csv('SeqNetworkLandscapeAminoAcidIdenticalAASeq2.csv')

In [43]:
df2.to_csv('SeqNetworkLandscapeIdenticalAA.csv')

In [55]:
df['StartCodon'].value_counts()

StartCodon
AGGAGGATATTGAGG    194433
AGGAGGAGTTTGAGG    192800
CGACGAATCCTCCGA    147295
CGACGATCCCTCCGA    138217
Name: count, dtype: int64

In [60]:
def uniqueAA(ntlist):
    AAlist=[]
    for i in ntlist:
        peptide=CommonMotifProb.translateRNA(i)
        if peptide not in AAlist:
            AAlist.append(peptide)
    return AAlist

In [57]:
#AGGAGGATATTGAGG
#CGACGAATCCTCCGA
df2=df[df['StartCodon']=='CGACGAATCCTCCGA']
df3=df[df['StartCodon']=='AGGAGGATATTGAGG']


In [64]:
alltargetCodons

array(['CGACGAATCCTCCGA', 'CGTCGAATCCTCCGA', 'CGACGAGTCCTCCGA',
       'CGACGAATCCGCCGA', 'CGACGAATCCTCCGG', 'CGACGAATCCTCCTA',
       'CGGCGAATCCTCCGA', 'CGACGGATCCTCCGA', 'CGATGAATCCTCCGA',
       'CGACGAATCATCCGA', 'CGACGAATCCTCTGA', 'CGACGATTCCTCCGA',
       'CGACGACTCCTCCGA', 'CGACAAATCCTCCGA', 'CGACGAATCCCCCGA',
       'CGGCGAATCCTCAGA', 'CGACGAATCCTTCGA', 'CAACGAATCCTCCGA',
       'CGACGAAGCCTCCGA', 'CGGCGAATCCTCCTA', 'CGACGAACCCTCCGA',
       'CGACGAATCCTCCAA', 'CGACGAACCCGCCGA', 'CGACGAATCTTCCGA',
       'CCACGAATCCGCCGA', 'CGACGAATCCTCCCA', 'CGACTAATCCTCCGA',
       'CGACGAATCCACCGA', 'CGACGAATCCTCAGA', 'CGACGACTCCCCCGA',
       'CGACGGATCCCCCGA', 'CGACGAATCTCCCGA', 'CGACGAAACCCCCGA',
       'CGCCGAATCCCCCGA', 'CTACGAATCCTCCGA', 'CCACGAATCCCCCGA',
       'CGACGAATTCCCCGA', 'CGACGAGTCCCCCGA', 'CGACGAATCCCCCTA',
       'CGACGAATCCCCCGG', 'CCACGAATCCTCCGA', 'CGACGAACCCCCCGA',
       'CGACGAATACTCCGA', 'CGACGAATTCTCCGA', 'CGACGAATGCTCCGA',
       'CGACGCATCCTCCGA', 'TGACGAATCCTCC

In [76]:
#ilestart='AGGAGGATATTGAGG'
#argstart='CGACGAATCCTCCGA'

ilestart='CGACGAATCCTCCGA'
argstart='AGGAGGATATTGAGG'

fullDF=df[(df['StartCodon']==ilestart) | (df['StartCodon']==argstart)]

alltargetCodons=fullDF.TargetCodon.unique()



columns=['NodeSeq','ParentSeq','Count','Motif']


motif1=createmotif([['R'],['R'],['S'],['L'],['R']])
datalist=[]



tempDF=fullDF


for TCodon in alltargetCodons:
    #print TCodon
    TcodonCount=0
    if TCodon!=ilestart and TCodon != argstart:
        protseq=CommonMotifProb.translateRNA(TCodon)
        ismotif=ismotifmatch(protseq,motif1)

        TcodonCount+=int(sum(tempDF[tempDF['TargetCodon']==TCodon]['Count']))

        if ismotif:
            motifValue=1
        else:
            motifValue=0

        if SeqMatch(ilestart,TCodon)==SeqMatch(argstart,TCodon):
            parentseq=3
        elif SeqMatch(ilestart,TCodon)<SeqMatch(argstart,TCodon):
            parentseq=1
        elif SeqMatch(ilestart,TCodon)>SeqMatch(argstart,TCodon):
            parentseq=2
        else:
            parentseq=0


        datalist.append([protseq,parentseq,TcodonCount,motifValue])

            
motifmutDF=pd.DataFrame(datalist,columns=columns)

In [50]:

newmotifmutdata=[]
alreadychecked=[]
for i,row in motifmutDF.iterrows():
    if row.NodeSeq not in alreadychecked and row.NodeSeq != 'RRILR':
        dftemp=motifmutDF[(motifmutDF['NodeSeq']==row.NodeSeq)]
        probs=dftemp['Count']
        probssum=probs.sum()
        newmotifmutdata.append([row.NodeSeq, row.ParentSeq, probssum,row.Motif])
        alreadychecked.append(row.NodeSeq)
            
newmotifmutDF=pd.DataFrame(newmotifmutdata,columns=columns)

In [52]:
motifs=motifmutDF[motifmutDF['Motif']==1]

one= motifs[motifs['ParentSeq']==1]
two= motifs[motifs['ParentSeq']==2]


In [53]:
motifs=motifmutDF[motifmutDF['Motif']==1]

one= motifs[motifs['ParentSeq']==1]
two= motifs[motifs['ParentSeq']==2]


In [54]:
newmotifmutDF.to_csv('NetworkDataIdenticalAA_DetailsAminoAcidSeq2.csv')